In [1]:
import os
import glob

import pyvista as pv

In [2]:
CLIP_BOUNDS = (-2, 4, -1.5, 1.5, 0.5, 0.5)

SLICE_NORMAL = (0, 0, 1)
SLICE_ORIGIN = (0, 0, 0.5)

INTERNAL_FIELDS = ["p", "U", "nut", "implicit_distance"]
AERO_FIELDS     = ["p", "U", "nut", "Normals"]   # “Normals” is what PyVista calls the computed normals
FREE_FIELDS     = ["p", "U", "nut"]

internal_path = "/local00/bioinf/airfrans/Simulations/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/VTK/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32_20000/internal.vtu"
aerofoil_path = "/local00/bioinf/airfrans/Simulations/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/VTK/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32_20000/boundary/aerofoil.vtp"
freestream_path = "/local00/bioinf/airfrans/Simulations/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/VTK/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32_20000/boundary/freestream.vtp"

internal_saving_path = "/system/user/publicdata/airfrans/Dataset_preprocessed/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/internal.vtu"
aerofoil_saving_path = "/system/user/publicdata/airfrans/Dataset_preprocessed/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/aerofoil.vtp"
freestream_saving_path = "/system/user/publicdata/airfrans/Dataset_preprocessed/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/freestream.vtp"


In [3]:
def select_fields(ds, names):
    """Return a copy of `ds` carrying *only* the arrays in `names`."""
    out = ds.copy()
    # point‐data
    for arr in list(out.point_data.keys()):
        if arr not in names:
            out.point_data.remove(arr)
    # cell‐data
    for arr in list(out.cell_data.keys()):
        if arr not in names:
            out.cell_data.remove(arr)
    return out

In [4]:
internal = pv.read(internal_path)
internal

UnstructuredGrid (0x7f786ad8bdc0)
  N Cells:    276860
  N Points:   556420
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   0.000e+00, 1.000e+00
  N Arrays:   9

In [5]:
# clip box
CLIP_BOUNDS = (-2, 4, -1.5, 1.5, 0, 1)

internal = internal.clip_box(bounds=CLIP_BOUNDS, crinkle=True, progress_bar=True, invert=False)  # clip box
internal

Clipping a Dataset by a Bounding Box: 100%|██████████[00:54<00:00]


UnstructuredGrid (0x7f786ac65fc0)
  N Cells:    176178
  N Points:   354832
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   0.000e+00, 1.000e+00
  N Arrays:   12

In [6]:
# slice to 2D
SLICE_NORMAL = (0, 0, 1)
SLICE_ORIGIN = (0, 0, 0.5)

internal = internal.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)
internal

Slicing: 100%|██████████[00:02<00:00]


PolyData (0x7f786ac1bca0)
  N Cells:    176178
  N Points:   177416
  N Strips:   0
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   11

In [7]:
aerofoil = pv.read(aerofoil_path)
aerofoil

PolyData (0x7f786ac180a0)
  N Cells:    994
  N Points:   1988
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   0.000e+00, 1.000e+00
  N Arrays:   9

In [8]:
# compute sdf to airfoil
internal = internal.compute_implicit_distance(aerofoil)
internal

PolyData (0x7f786ac65f00)
  N Cells:    176178
  N Points:   177416
  N Strips:   0
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   12

In [9]:
# only keep necessary fields
INTERNAL_FIELDS = ["p", "U", "nut", "implicit_distance"]

internal = select_fields(internal, INTERNAL_FIELDS)
internal

PolyData (0x7f786ad8bfa0)
  N Cells:    176178
  N Points:   177416
  N Strips:   0
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   8

In [11]:
pv.save_meshio("internal.vtu", internal)

In [12]:
internal

PolyData (0x7f786ad8bfa0)
  N Cells:    176178
  N Points:   177416
  N Strips:   0
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   8

In [14]:
internal_new = pv.read("internal.vtu")
internal_new

UnstructuredGrid (0x7f785d652fe0)
  N Cells:    176178
  N Points:   177416
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   7

In [111]:
aerofoil = aerofoil.compute_normals(point_normals=True, cell_normals=True, flip_normals=False)  # normals
aerofoil

PolyData (0x7fbcd76e8d60)
  N Cells:    994
  N Points:   1988
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   0.000e+00, 1.000e+00
  N Arrays:   11

In [112]:
aerofoil = aerofoil.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)
aerofoil

Slicing: 100%|██████████[00:00<00:00]


PolyData (0x7fbcd76e9840)
  N Cells:    994
  N Points:   994
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   11

In [113]:
# only keep necessary fields
AERO_FIELDS = ["p", "U", "nut", "Normals"]   # “Normals” is what PyVista calls the computed normals

aerofoil = select_fields(aerofoil, AERO_FIELDS)
aerofoil

PolyData (0x7fbcdd062680)
  N Cells:    994
  N Points:   994
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   9

In [15]:
aerofoil.save("aerofoil.vtp")

In [16]:
aerofoil

PolyData (0x7f786ac180a0)
  N Cells:    994
  N Points:   1988
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   0.000e+00, 1.000e+00
  N Arrays:   9

In [114]:
aerofoil = aerofoil.compute_cell_sizes(area=False, volume=False)
aerofoil

PolyData (0x7fbcd76eaf20)
  N Cells:    994
  N Points:   994
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   10

In [115]:
freestream = pv.read(freestream_path)
freestream

PolyData (0x7fbcd779dde0)
  N Cells:    1706
  N Points:   3412
  N Strips:   0
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   0.000e+00, 1.000e+00
  N Arrays:   9

In [116]:
freestream = freestream.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)  # slice to 2D
freestream

Slicing: 100%|██████████[00:00<00:00]


PolyData (0x7fbcd779d9c0)
  N Cells:    1706
  N Points:   1706
  N Strips:   0
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   9

In [117]:
freestream = select_fields(freestream, FREE_FIELDS)  # only keep necessary fields
freestream

PolyData (0x7fbcd76eb040)
  N Cells:    1706
  N Points:   1706
  N Strips:   0
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   7

In [ ]:
# params
CLIP_BOUNDS = (-2, 4, -1.5, 1.5, 0, 1)
SLICE_NORMAL = (0, 0, 1)
SLICE_ORIGIN = (0, 0, 0.5)

INTERNAL_FIELDS = ["p", "U", "nut", "implicit_distance"]
AERO_FIELDS = ["p", "U", "nut", "Normals"]
FREE_FIELDS = ["p", "U", "nut"]


# load raw files
internal = pv.read(internal_path)
aerofoil = pv.read(aerofoil_path)
freestream = pv.read(freestream_path)

# internal preprocessing
internal = internal.clip_box(bounds=CLIP_BOUNDS, crinkle=True, progress_bar=True, invert=False)  # clip box
internal = internal.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)  # slice to 2D
internal = internal.compute_implicit_distance(aerofoil)  # compute sdf to airfoil
internal = select_fields(internal, INTERNAL_FIELDS)  # only keep necessary fields

# aerofoil preprocessing
aerofoil = aerofoil.compute_normals(point_normals=True, cell_normals=True, flip_normals=False)  # normals
aerofoil = aerofoil.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)  # slice to 2D
aerofoil = select_fields(aerofoil, AERO_FIELDS)  # only keep necessary fields
aerofoil = aerofoil.compute_cell_sizes(area=False, volume=False)  # compute length (arc length)

# freestream processing
freestream = freestream.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)  # slice to 2D
freestream = select_fields(freestream, FREE_FIELDS)  # only keep necessary fields


(177416, 3)

In [5]:
internal = pv.read(internal_path)
aerofoil = pv.read(aerofoil_path)
freestream = pv.read(freestream_path)

# internal preprocessing
internal2D = internal.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)
internal_clipped = internal2D.clip_box(bounds=CLIP_BOUNDS, crinkle=True, progress_bar=True)  # clip box
internal = internal_clipped.compute_implicit_distance(surface=aerofoil)  # distance function to aerofoil surface
# internal = select_fields(internal, INTERNAL_FIELDS)
# TODO: add normals = 0 for points not on the surface (or is this handled by the Simulation class?)
pv.save_meshio(internal_saving_path, internal_clipped)

# aerofoil preprocessing
aerofoil = aerofoil.compute_normals(point_normals=True, cell_normals=False, flip_normals=False)  # normals
aerofoil = aerofoil.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)
# aerofoil = select_fields(aerofoil, AERO_FIELDS)
aerofoil.save(aerofoil_saving_path)

# freestream preprocessing
freestream = freestream.slice(normal=SLICE_NORMAL,
                      origin=SLICE_ORIGIN,
                      generate_triangles=False,
                      progress_bar=True)
# freestream = select_fields(freestream, FREE_FIELDS)
freestream.save(freestream_saving_path)

# TODO: add field of on surface or not (or is this handled by the Simulation class??)


Slicing: 100%|██████████[00:59<00:00]
Clipping a Dataset by a Bounding Box: 100%|██████████[00:52<00:00]
Slicing: 100%|██████████[00:00<00:00]
Slicing: 100%|██████████[00:00<00:00]


In [118]:
internal_new = pv.read(internal_saving_path)
aerofoil_new = pv.read(aerofoil_saving_path)
freestream_new = pv.read(freestream_saving_path)

internal_old = pv.read("/system/user/publicdata/airfrans/Dataset/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32_internal.vtu")
aerofoil_old = pv.read("/system/user/publicdata/airfrans/Dataset/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32_aerofoil.vtp")
freestream_old = pv.read("/system/user/publicdata/airfrans/Dataset/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32/airFoil2D_SST_31.283_-4.156_0.919_6.98_14.32_freestream.vtp")
 

In [119]:
internal_new

UnstructuredGrid (0x7fbcd76ea3e0)
  N Cells:    102162
  N Points:   103754
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   11

In [120]:
internal_old

UnstructuredGrid (0x7fbcd76e9de0)
  N Cells:    176178
  N Points:   177416
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   11

In [121]:
internal

PolyData (0x7fbcd76eaec0)
  N Cells:    176178
  N Points:   177416
  N Strips:   0
  X Bounds:   -2.159e+00, 4.226e+00
  Y Bounds:   -1.616e+00, 1.617e+00
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   8

In [122]:
aerofoil_old

PolyData (0x7fbcd779d480)
  N Cells:    994
  N Points:   994
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   10

In [123]:
aerofoil

PolyData (0x7fbcd76eaf20)
  N Cells:    994
  N Points:   994
  N Strips:   0
  X Bounds:   -7.739e-06, 1.000e+00
  Y Bounds:   -6.562e-02, 7.800e-02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   10

In [124]:
freestream_old

PolyData (0x7fbcd76eb5e0)
  N Cells:    1706
  N Points:   1706
  N Strips:   0
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   7

In [125]:
freestream

PolyData (0x7fbcd76eb040)
  N Cells:    1706
  N Points:   1706
  N Strips:   0
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   5.000e-01, 5.000e-01
  N Arrays:   7